# <font color=#ffa500>Projeto Webnb - Extração dos Dados</font>

### <font color=#ffa500>Membros: Carolina Hirschheimer, Paulo Kim, Raphael Lahiry</font>

# <font color=#ffa500>Bibliotecas utilizadas</font>

In [10]:
# Bibliotecas úteis 
import numpy as np
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from tqdm import tqdm

# <font color=#ffa500>Extração dos Dados</font>

In [4]:
# extração dos dados desejados através do web scraping do Airbnb

url = 'https://www.airbnb.com.br/s/S%C3%A3o-Paulo-~-SP/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=july&flexible_trip_dates%5B%5D=june&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&query=S%C3%A3o%20Paulo%20-%20SP&place_id=ChIJ0WGkg4FEzpQRrlsz_whLqZs&disable_auto_translation=false&source=structured_search_input_header&search_type=autocomplete_click'
resposta = requests.get(url = url)
resposta.encoding = 'utf-8'

lista_titulo = []
lista_bairro = []
lista_imovel = []
lista_preco = []
lista_avaliacao = []
lista_caracteristicas = []
    
for _ in tqdm(range(14), desc = 'Extraindo dados...'):    

    soup = BeautifulSoup(resposta.text, 'html.parser')
    
    anuncios = soup.findAll('div', class_='_8ssblpx') # anúncios

    for i in range(0, len(anuncios)):
        texto_caracteristicas = []

        anuncio = anuncios[i]

        titulo = anuncio.find('meta', itemprop='name').attrs['content']
        lista_titulo.append(titulo)

        bairro = anuncio.find('div', class_="_b14dlit").get_text().split(' em ')[1]
        lista_bairro.append(bairro)

        tipo_imovel = anuncio.find('div', class_="_b14dlit").get_text().split(' em ')[0]
        lista_imovel.append(tipo_imovel)

        caracteristicas = anuncio.findAll('span', class_="_3hmsj")[2:]
        for caracteristica in caracteristicas:
            texto_caracteristicas.append(caracteristica.get_text())
        lista_caracteristicas.append(texto_caracteristicas)

        preco = int(anuncio.find('span', class_="_155sga30").get_text()[2:])
        lista_preco.append(preco)

        avaliacao = anuncio.find('span', class_="_10fy1f8")
        if avaliacao is not None:
            avaliacao = float(anuncio.find('span', class_="_10fy1f8").get_text())
        else:
            avaliacao = np.nan
        lista_avaliacao.append(avaliacao)
        
    url = 'https://www.airbnb.com.br' + soup.find('a', class_="_za9j7e").attrs['href'] # próxima página
    resposta = requests.get(url = url)
    resposta.encoding = 'utf-8'

Extraindo dados...: 100%|██████████████████████████████████████████████████████████████| 14/14 [00:32<00:00,  2.33s/it]


# <font color=#ffa500>Dataframe</font>

In [5]:
dados = pd.DataFrame({'Título': lista_titulo,
                      'Preço (R$)': lista_preco,
                      'Avaliação': lista_avaliacao,
                      'Bairro': lista_bairro,
                      'Imóvel': lista_imovel,
                      'Características': lista_caracteristicas
                     })

In [11]:
dados.head()

,Título,Preço (R$),Avaliação,Bairro,Imóvel,Características
0,FLAT PARTICULAR NO MELIA - AV. IBIRAPUERA,110,4.90,Indianópolis,Flat inteiro,"[2 hóspedes, 1 quarto, 1 cama, 1 banheiro, Wi-..."
1,Studio novinho a 300 m metrô - Self check in,117,4.80,Consolacao,Apartamento inteiro,"[5 hóspedes, Estúdio, 3 camas, 1 banheiro, Wi-..."
2,Studio encantador em São Paulo próximo ao metrô,110,4.87,Consolacao,Flat inteiro,"[2 hóspedes, 1 quarto, 1 cama, 1 banheiro, Wi-..."
3,Flat mobiliado ao lado do Hospital Sirio Libanês,80,4.61,Bela Vista,Flat inteiro,"[2 hóspedes, 1 quarto, 1 cama, 1 banheiro, Wi-..."
4,Suíte moderna com cama de casal no Brooklin,54,4.89,Vila Olímpia,Quarto inteiro,"[2 hóspedes, 1 quarto, 1 cama, 1 banheiros pri..."


In [7]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Título           350 non-null    object 
 1   Preço (R$)       350 non-null    int64  
 2   Avaliação        297 non-null    float64
 3   Bairro           350 non-null    object 
 4   Imóvel           350 non-null    object 
 5   Características  350 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 16.5+ KB


# <font color=#ffa500>Salvando dados em um arquivo csv</font>

In [8]:
#dados.to_csv('dados_anuncios_airbnb.csv', index=False)